In [0]:
import sys
sys.path.append("/Workspace/Users/mandu543@gmail.com/Movies_Project")

from src.config import SILVER_TABLE, GOLD_TABLE

# Liste tous les schemas du catalog 'workspace'
spark.sql("SHOW SCHEMAS IN workspace").show()

# Create schema if not exists
spark.sql(
    f"CREATE SCHEMA IF NOT EXISTS {GOLD_TABLE.split('.')[1]}"
)


In [0]:
# -----------------------------
# Créer ou remplacer la table Gold en filtrant certaines colonnes
# -----------------------------
try:
    # Lire la table Silver
    df_silver = spark.table(SILVER_TABLE)
    
    # Sélectionner toutes les colonnes sauf celles à exclure
    cols_to_exclude = ["backdrop_path", "imdb_id", "poster_path", "keywords"]
    cols_to_keep = [c for c in df_silver.columns if c not in cols_to_exclude]
    df_gold = df_silver.select(*cols_to_keep)
    
    # Écrire dans la table Gold
    df_gold.write.format("delta").mode("overwrite").saveAsTable(GOLD_TABLE)
    
    # Optimiser pour BI
    spark.sql(f"OPTIMIZE {GOLD_TABLE} ZORDER BY release_date")
    
    print(f"✅ Gold table {GOLD_TABLE} ready with selected columns")

except Exception as e:
    print(f"❌ Erreur lors de la création de la table Gold : {e}")